# Llama2 LangChain Output Parsers  

 > Fixing common errors from Llama2 outputs that do not conform to standards set by OpenAI. 

In [ ]:
#| default_exp utils.output_parsers
from nbdev import *

In [1]:
#| export
import asyncio

import json

from langchain.callbacks.base import BaseCallbackHandler
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chat_models.ollama import ChatOllama
from langchain.llms import Ollama
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema import AgentAction, AgentFinish, HumanMessage, AIMessage
from langchain.schema import BaseMessage, BaseOutputParser, OutputParserException
from langchain.schema import LLMResult
from langchain.utils.input import print_text

import os
import re
import sys
from typing import (
    Any,
    Dict,
    List,
    Union,
    Optional
)

import json
import re
from typing import Union

from langchain.agents.agent import AgentOutputParser
from langchain.agents.chat.prompt import FORMAT_INSTRUCTIONS
from langchain.schema import AgentAction, AgentFinish, OutputParserException
from langchain.output_parsers.json import parse_json_markdown


In [1]:
#| export

class JsonEnclosedByTextOutputParser(BaseOutputParser[Any]):
    """Parse the output of an LLM call to a JSON object."""

    def parse(self, text: str) -> Any:
        text = text.strip()
        m = re.search('.*([\[\{](.|\n)*[\}\]]).*', text, flags=re.M)
        if m:
            text1 = m.group(1)

            # need to make sure all entries in the JSON are quoted
            # so that the JSON parser can parse it
            # e.g. {"a": E} -> {"a": "E"}
            # this is a hack, but it works
            text2 = re.sub(r':\s*([a-zA-Z0-9_]+)\s*([,\]\}])', r': "\1"\2', text1)
            try:
                return json.loads(text2)
            except json.JSONDecodeError as e:
                raise OutputParserException(f"Invalid json output: {text2} derived from {text}") from e
        else: 
            raise OutputParserException(f"Could not find json-formatted data in: {text}")

    @property
    def _type(self) -> str:
        return "json_enclosed_by_text_output_parser"
    


NameError: name 'BaseOutputParser' is not defined

In [ ]:
#| export

FINAL_ANSWER_ACTION = "Final Answer:"

class ReActJsonSingleInputOutputParser_llama2(AgentOutputParser):
    """Parses ReAct-style LLM calls that have a single tool input in json format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken,
    should be in the below format. This will result in an AgentAction
    being returned.

    ```
    Thought: agent thought here
    Action:
    ```
    {
        "action": "search",
        "action_input": "what is the temperature in SF"
    }
    ```
    ```

    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish
    being returned.

    ```
    Thought: agent thought here
    Final Answer: The temperature is 100 degrees
    ```

    """

    #pattern = re.compile(r"^.*?`{3}(?:json)?\n(.*?)`{3}.*?$", re.DOTALL)
    pattern = re.compile('.*([\[\{].*[\}\]]).*', re.DOTALL)
    """Regex pattern to parse the output."""

    def get_format_instructions(self) -> str:
        return FORMAT_INSTRUCTIONS

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        includes_answer = FINAL_ANSWER_ACTION in text
        try:
            found = self.pattern.search(text)
            if not found:
                # Fast fail to parse Final Answer.
                raise ValueError("action not found")
            action = found.group(1)
            action_clean = re.sub(r':\s*([a-zA-Z0-9_]+)\s*([,\]\}])', r': "\1"\2', action)
            response = json.loads(action_clean.strip())
            includes_action = "action" in response
            if includes_answer and includes_action:
                raise OutputParserException(
                    "Parsing LLM output produced a final answer "
                    f"and a parse-able action: {text}"
                )
            return AgentAction(
                response["action"], response.get("action_input", {}), text
            )

        except Exception:
            if not includes_answer:
                raise OutputParserException(f"Could not parse LLM output: {text}")
            output = text.split(FINAL_ANSWER_ACTION)[-1].strip()
            return AgentFinish({"output": output}, text)

    @property
    def _type(self) -> str:
        return "react-json-single-input"

class JSONAgentOutputParser_llama2(AgentOutputParser):
    """Parses tool invocations and final answers in JSON format.

    Expects output to be in one of two formats.

    If the output signals that an action should be taken,
    should be in the below format. This will result in an AgentAction
    being returned.

    ```
    {
      "action": "search",
      "action_input": "2+2"
    }
    ```

    If the output signals that a final answer should be given,
    should be in the below format. This will result in an AgentFinish
    being returned.

    ```
    {
      "action": "Final Answer",
      "action_input": "4"
    }
    ```
    """

    def parse(self, text: str) -> Union[AgentAction, AgentFinish]:
        try:
            response = parse_json_markdown(text)
            if isinstance(response, list):
                # gpt turbo frequently ignores the directive to emit a single action
                logger.warning("Got multiple action responses: %s", response)
                response = response[0]
            if response["action"] == "Final Answer":
                return AgentFinish({"output": response["action_input"]}, text)
            else:
                return AgentAction(
                    response["action"], response.get("action_input", {}), text
                )
        except Exception as e:
            raise OutputParserException(f"Could not parse LLM output: {text}") from e

    @property
    def _type(self) -> str:
        return "json-agent"